In [1]:
import cplex
import cobra
import time
import pickle
cobra.Configuration().solver='cplex'

## 1. Selección de modelos de tejido y de mieloma

### Codigo empleado para cargar por primera vez los modelos y guardarlos con Pickle

In [34]:
#start=time.time()
#blood=cobra.io.read_sbml_model("../Entrega Paco/blood.xml")
#print(time.time()-start)

75.93215274810791


In [35]:
#start=time.time()
#liver=cobra.io.read_sbml_model("../Entrega Paco/liver.xml")
#print(time.time()-start)

80.54548740386963


In [36]:
#start=time.time()
#muscle=cobra.io.read_sbml_model("../Entrega Paco/muscle.xml")
#print(time.time()-start)

75.86897373199463


In [37]:
#start=time.time()
#KMS20=cobra.io.read_sbml_model("../Entrega Paco/KMS20.xml")
#print(time.time()-start)

76.25639605522156


In [38]:
#start=time.time()
#HUNS1=cobra.io.read_sbml_model("../Entrega Paco/HUNS1.xml")
#print(time.time()-start)

73.2776792049408


In [39]:
#start=time.time()
#JJN3=cobra.io.read_sbml_model("../Entrega Paco/JJN3.xml")
#print(time.time()-start)

75.16148519515991


In [40]:
#import pickle #Para guardar información (en formato comprimido)

#nombre="./blood.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(blood, f)
    
#nombre="./liver.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(liver, f)

#nombre="./muscle.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(muscle, f)

#nombre="./KMS20.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(KMS20, f)

#nombre="./HUNS1.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(HUNS1, f)

#nombre="./JJN3.dat"
#with open(nombre, "wb") as f:
#    pickle.dump(JJN3, f)

### Carga de los Modelos con Pickle

In [2]:
nombre="./blood.dat"
with open(nombre, "rb") as f:
    blood = pickle.load(f)

nombre="./liver.dat"
with open(nombre, "rb") as f:
    liver = pickle.load(f)

nombre="./muscle.dat"
with open(nombre, "rb") as f:
    muscle = pickle.load(f)
    
nombre="./KMS20.dat"
with open(nombre, "rb") as f:
    KMS20 = pickle.load(f)

nombre="./HUNS1.dat"
with open(nombre, "rb") as f:
    HUNS1 = pickle.load(f)
    
nombre="./JJN3.dat"
with open(nombre, "rb") as f:
    JJN3 = pickle.load(f)

nombre="./Human_1_17.dat"
with open(nombre, "rb") as f:
    human = pickle.load(f)

**Para llevar a cabo tanto el punto 2 como el punto 3 he utilizado un script adicional al que he llamado "carga_gCSs.ipynb", el cual me permite calcular las toxicidades genéticas específicas de cada tejido sano, así como los targets genéticos de cada modelo de mieloma ("cut-sets" de cada modelo), ejecutando dicho script desde la máquina "Ampere"**

**Una vez he calculado todos los genes para cada modelo, los guardo en sus respectivos archivos y los cargo posteriormente otra vez en Pascal mediante Pickle, que es lo que se puede ver a continuación**

## Carga de los "Cut-sets"

In [3]:
nombre="./bloodgMCS.dat"
with open(nombre, "rb") as f:
    bloodgMCS=pickle.load(f)
len(bloodgMCS)

133

In [4]:
nombre="./livergMCS.dat"
with open(nombre, "rb") as f:
    livergMCS=pickle.load(f)
len(livergMCS)

131

In [5]:
nombre="./musclegMCS.dat"
with open(nombre, "rb") as f:
    musclegMCS=pickle.load(f)
len(musclegMCS)

120

In [6]:
nombre="./JJN3gMCS.dat"
with open(nombre, "rb") as f:
    JJN3gMCS=pickle.load(f)
len(JJN3gMCS)

105

In [7]:
nombre="./KMS20gMCS.dat"
with open(nombre, "rb") as f:
    KMS20gMCS=pickle.load(f)
len(KMS20gMCS)

122

In [8]:
nombre="./HUNS1gMCS.dat"
with open(nombre, "rb") as f:
    HUNS1gMCS=pickle.load(f)
len(HUNS1gMCS)

112

In [9]:
# Toxicidades del modelo genérico
nombre="./humangMCS.dat"
with open(nombre, "rb") as f:
    humangMCS=pickle.load(f)
len(humangMCS)

92

**En todas estas variables de encima están almacenados los targets específicos para cada tejido/modelo**

## 4. Elimina de los conjuntos anteriores aquellos genes que sean toxicidades genéricas

**A continuación, lo siguiente que hacemos es, dentro de todos esos genes, eliminar en cada caso aquellos genes que sean "toxicidades genéricas". Para ello hacemos uso de "humangMCS", donde están almacenados los targets genéricos.**

In [9]:
EspecificasSangre = bloodgMCS.difference(humangMCS)
len(EspecificasSangre)

41

In [10]:
EspecificasHígado = livergMCS.difference(humangMCS)
len(EspecificasHígado)

39

In [11]:
EspecificasMusculo = musclegMCS.difference(humangMCS)
len(EspecificasMusculo)

28

**Para saber cuales son las toxicidades específicas de cada mieloma, tenemos que hacer la diferencia entre el modelo genérico (human) y los diferentes modelos de mieloma.**

In [12]:
EspecificasKMS20 = KMS20gMCS.difference(humangMCS)
len(EspecificasKMS20)

31

In [13]:
EspecificasHUNS1 = HUNS1gMCS.difference(humangMCS)
len(EspecificasHUNS1)

21

In [14]:
EspecificasJJN3 = JJN3gMCS.difference(humangMCS)
len(EspecificasJJN3)

14

## Punto 5 y 6. Estudio sobre que estrategias son mejores para eliminar líneas de mieloma múltiple, y análisis de algunas de estas estrategias

### Ahora, vamos a estudiar cuales podrían ser en teoría los mejores targets, para ellos vamos a cruzar las toxicidades específicas obtenidas para los diferentes modelos de tejido sano y los diferentes modelos de mieloma

### Modelo de mieloma "KMS20"

Para eliminar a las células de mieloma, lo más sencillo es eliminar aquellos genes que no sean genéricos y actuen como targets de cada modelo de mieloma, que es lo que hemos realizado justo en el código anterior. Sin emabrgo, lo ideal es que dichos genes no solo no sean targets genéricos, si no que tampoco afecten a otros tejidos más concretos como en nuestro caso pueden ser la sangre, el hígado o los músculos.

In [44]:
KMS20_Nivel_2 = KMS20gMCS.difference(humangMCS).difference(bloodgMCS)
len(KMS20_Nivel_2)

13

Vemos como se obtienen muchos menos genes cuando buscamos genes que tampoco afecten a la sangre.

Ahora vamos a comprobar si existe algun target el cual no afecte a ninguno de los 3 tejidos que hemos seleccionado para esta entrega, lo cual sería lo ideal, ya que podríamos eliminar las células de mieloma sin afectar a ninguno de nustros tejidos sanos.

In [17]:
KMS20_Nivel_1 = KMS20gMCS.difference(humangMCS).difference(bloodgMCS).difference(musclegMCS).difference(livergMCS)
len(KMS20_Nivel_1)

3

In [18]:
KMS20_Nivel_1

{'ENSG00000111726', 'ENSG00000137700', 'ENSG00000164414'}

Vemos como hay 3 posibles genes: 'ENSG00000111726', 'ENSG00000137700' y 'ENSG00000164414'.

Para comprobar que estos genes realmente son targets, hacemos una simulación del modelo eliminando dichos genes:

In [28]:
for g in KMS20_Nivel_1:
    with human:
        human.genes.get_by_id(g).knock_out()
        sol1=human.slim_optimize()
    with blood:
        blood.genes.get_by_id(g).knock_out()
        sol2=blood.slim_optimize()
    with liver:
        liver.genes.get_by_id(g).knock_out()
        sol3=liver.slim_optimize()
    with muscle:
        liver.genes.get_by_id(g).knock_out()
        sol4=muscle.slim_optimize()
    with KMS20:
        KMS20.genes.get_by_id(g).knock_out()
        sol5=KMS20.slim_optimize()
    print(g,sol1,sol2,sol3,sol4,sol5)

ENSG00000164414 124.7607520981648 81.56232786610786 78.60304815473107 86.77338693646946 0.0
ENSG00000111726 124.7607520981648 81.56232786610786 78.60304815473107 86.77338693646946 0.0


KeyError: 'ENSG00000137700'

Vemos como en 2 de los casos, se puede apreciar claramente como el nivel de producción de biomasa en todos los tejidos es mayor que 0, excepto en el modelo de mieloma, donde la producción de biomasa es 0, lo que indica que estos genes si que son targets específicos de dicho mieloma y no afectan a nuestros tejidos.

Por otro lado, el error que se obtiene se debe a que el gen en concreto que da error ("ENSG00000137700") no está presente en alguno de los modelos de tejido que estamos utilizando, algo que puede ocurrir perfectamente.

### Modelo de mieloma "HUNS1"

In [45]:
HUNS1_Nivel_2 = HUNS1gMCS.difference(humangMCS).difference(bloodgMCS)
len(HUNS1_Nivel_2)

8

In [18]:
HUNS1_Nivel_1 = HUNS1gMCS.difference(humangMCS).difference(bloodgMCS).difference(musclegMCS).difference(livergMCS)
len(HUNS1_Nivel_1)

5

In [19]:
HUNS1_Nivel_1

{'ENSG00000134184',
 'ENSG00000135437',
 'ENSG00000137700',
 'ENSG00000156136',
 'ENSG00000173614'}

En el caso del modelo de mieloma "HUNS1", hay 5 genes los cuales pueden servirnos como targets y no afectarían a ninguno de los 3 tejidos que hemos seleccionado, además de claro está no afectar a ninguno de los genes genéricos. Estos genes son: 'ENSG00000134184', 'ENSG00000135437', 'ENSG00000137700', 'ENSG00000156136' y 'ENSG00000173614'.

In [29]:
for g in HUNS1_Nivel_1:
    with human:
        human.genes.get_by_id(g).knock_out()
        sol1=human.slim_optimize()
    with blood:
        blood.genes.get_by_id(g).knock_out()
        sol2=blood.slim_optimize()
    with liver:
        liver.genes.get_by_id(g).knock_out()
        sol3=liver.slim_optimize()
    with muscle:
        liver.genes.get_by_id(g).knock_out()
        sol4=muscle.slim_optimize()
    with HUNS1:
        HUNS1.genes.get_by_id(g).knock_out()
        sol5=HUNS1.slim_optimize()
    print(g,sol1,sol2,sol3,sol4,sol5)

ENSG00000135437 124.7607520981648 81.56232786610786 78.60304815473107 86.77338693646946 0.0
ENSG00000173614 124.7607520981648 81.56232786610786 78.60304815473106 86.77338693646946 0.0
ENSG00000156136 124.7607520981648 81.56232786610786 78.60304815473106 86.77338693646946 0.0
ENSG00000134184 124.7607520981648 81.56232786610786 0.0 86.77338693646946 0.0


KeyError: 'ENSG00000137700'

### Modelo de mieloma "JJN3"

In [46]:
JJN3_Nivel_2 = JJN3gMCS.difference(humangMCS).difference(bloodgMCS)
len(JJN3_Nivel_2)

5

In [20]:
JJN3_Nivel_1 = JJN3gMCS.difference(humangMCS).difference(bloodgMCS).difference(musclegMCS).difference(livergMCS)
len(JJN3_Nivel_1)

2

In [21]:
JJN3_Nivel_1

{'ENSG00000137700', 'ENSG00000156136'}

En el caso del modelo de mieloma "JJN3" tan solo 2 genes podrían actuar como targets específicos sin afectar a ninguno de nuestros tejidos de interés. Dichos genes serían: 'ENSG00000137700' y 'ENSG00000156136'.

In [30]:
for g in JJN3_Nivel_1:
    with human:
        human.genes.get_by_id(g).knock_out()
        sol1=human.slim_optimize()
    with blood:
        blood.genes.get_by_id(g).knock_out()
        sol2=blood.slim_optimize()
    with liver:
        liver.genes.get_by_id(g).knock_out()
        sol3=liver.slim_optimize()
    with muscle:
        liver.genes.get_by_id(g).knock_out()
        sol4=muscle.slim_optimize()
    with JJN3:
        JJN3.genes.get_by_id(g).knock_out()
        sol5=JJN3.slim_optimize()
    print(g,sol1,sol2,sol3,sol4,sol5)

ENSG00000156136 124.7607520981648 81.56232786610786 0.0 86.77338693646946 0.0


KeyError: 'ENSG00000137700'

**Una vez analizado cada modelo de mieloma por separado, vamos a comprobar si alguno de los targets obtenidos en cada mieloma es común para todos los modelos de mieloma con los que estamos trabajando**

In [32]:
targets_comunes = KMS20_Nivel_1.intersection(HUNS1_Nivel_1).intersection(JJN3_Nivel_1)
print(targets_comunes)

{'ENSG00000137700'}


**Haciendo la intersección entre los 3 sets, descubrimos que hay 1 gen el cual actua como target genético en los 3 modelos de mieloma que estamos estudiando, el gen "ENSG00000137700". Si buscamos dicho gen en los modelos de mieloma que tenemos:**

In [48]:
KMS20.genes.ENSG00000137700

Gene identifier,ENSG00000137700
Name,
Memory address,0x07fe1379d2160
Functional,True
In 3 reaction(s),"MAR01621, MAR06386, MAR04856"


**Vemos como no nos dan mucha información acerca del gen aparte de que participa en 3 reacciones. Vamos a ver que reacciones son estas**

In [13]:
HUNS1.reactions.MAR01621

Reaction identifier,MAR01621
Name,"Phosphate Transport, Endoplasmic Reticulum"
Memory address,0x07fc5f6ec2e10
Stoichiometry,MAM02751r[r] <=> MAM02751c[c] Pi [Endoplasmic reticulum] <=> Pi [Cytosol]
GPR,ENSG00000137700
Lower bound,-1000.0
Upper bound,1000.0


In [50]:
KMS20.reactions.MAR06386

Reaction identifier,MAR06386
Name,
Memory address,0x07fe14333ae10
Stoichiometry,MAM01968c[c] + MAM02751r[r] --> MAM01968r[r] + MAM02751c[c] glucose-6-phosphate [Cytosol] + Pi [Endoplasmic reticulum] --> glucose-6-phosphate [Endoplasmic reticulum] + Pi [Cytosol]
GPR,ENSG00000137700
Lower bound,0.0
Upper bound,1000.0


In [51]:
KMS20.reactions.MAR04856

Reaction identifier,MAR04856
Name,alpha-D-Glucose 6-phosphate ketol-isomerase
Memory address,0x07fe14333a240
Stoichiometry,MAM01968c[c] <=> MAM01968r[r] glucose-6-phosphate [Cytosol] <=> glucose-6-phosphate [Endoplasmic reticulum]
GPR,ENSG00000105220 or ENSG00000137700
Lower bound,-1000.0
Upper bound,1000.0


**Analizando las 3 reacciones en las que participa este gen, vemos como se trata de 2 reacciones principalmente:**
- Transporte de Fosfato (Pi) desde el Retículo Endoplasmático al Citosol
- Transporte de Glucosa-6-Fosafato desde el Citosol al Retículo Endoplasmático 

**Vamos a comprobar que el gen "ENSG00000137700" es un target común para las 3 líneas de mieloma**

In [58]:
target = "ENSG00000137700"
with JJN3:
    JJN3.genes.get_by_id(target).knock_out()
    sol1=JJN3.slim_optimize()
with KMS20:
    KMS20.genes.get_by_id(target).knock_out()
    sol2=KMS20.slim_optimize()
with HUNS1:
    HUNS1.genes.get_by_id(target).knock_out()
    sol3=HUNS1.slim_optimize()
print(target,sol1,sol2,sol3)

ENSG00000137700 0.0 0.0 0.0


Vemos como la producción de Biomasa en los 3 modelos de mieloma es igual a 0, lo que significa que eliminando este gen, se podrían eliminar las células de los 3 tipos de mieloma que estamos estudiando en este caso.

**Si comparamos los modelos de mieloma por parejas:**

In [33]:
targets_comunes_KMS20_HUNS1 = KMS20_Nivel_1.intersection(HUNS1_Nivel_1)
targets_comunes_KMS20_JJN3 = KMS20_Nivel_1.intersection(JJN3_Nivel_1)
targets_comunes_HUNS1_JJN3 = HUNS1_Nivel_1.intersection(JJN3_Nivel_1)
print(targets_comunes_KMS20_HUNS1)
print(targets_comunes_KMS20_JJN3)
print(targets_comunes_HUNS1_JJN3)

{'ENSG00000137700'}
{'ENSG00000137700'}
{'ENSG00000156136', 'ENSG00000137700'}


**Podemos observar como solo en el caso de los modelos "HUNS1" y "JJN3" hay un target común diferente al obtenido anteriormente, que es el gen "ENSG00000156136".**

In [41]:
JJN3.genes.ENSG00000156136

Gene identifier,ENSG00000156136
Name,
Memory address,0x07fe12e19b4a8
Functional,True
In 6 reaction(s),"MAR08479, MAR04083, MAR08478, MAR08477, MAR04012, MAR04513"


In [14]:
JJN3.reactions.MAR08479

Reaction identifier,MAR08479
Name,
Memory address,0x07fc5fa082c88
Stoichiometry,MAM01644n[n] + MAM02039n[n] + MAM03106n[n] <=> MAM01668n[n] + MAM03130n[n] dCMP [Nucleus] + H+ [Nucleus] + UDP [Nucleus] <=> deoxycytidine [Nucleus] + UTP [Nucleus]
GPR,ENSG00000156136
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
JJN3.reactions.MAR04083

Reaction identifier,MAR04083
Name,ATP:adenosine 5'-phosphotransferase
Memory address,0x07fc5fa2e3b38
Stoichiometry,MAM01280m[m] + MAM01371m[m] --> MAM01285m[m] + MAM01334m[m] + MAM02039m[m] adenosine [Mitochondria] + ATP [Mitochondria] --> ADP [Mitochondria] + AMP [Mitochondria] + H+ [Mitochondria]
GPR,ENSG00000156110 or ENSG00000156136
Lower bound,0.0
Upper bound,1000.0


In [16]:
target = "ENSG00000156136"
with JJN3:
    JJN3.genes.get_by_id(target).knock_out()
    sol1=JJN3.slim_optimize()
with HUNS1:
    HUNS1.genes.get_by_id(target).knock_out()
    sol2=HUNS1.slim_optimize()
print(target,sol1,sol2)

ENSG00000156136 0.0 0.0


**Finalmente, estudiamos un target específico para la línea de mieloma "KMS20"**

Si recordamos, los targets específicos para este tipo de mieloma, y que no afectaban a ninguno de los tejidos sanos eran los siguientes:

In [19]:
KMS20_Nivel_1

{'ENSG00000111726', 'ENSG00000137700', 'ENSG00000164414'}

Sin tener en cuenta el gen "ENSG00000137700", que ya ha sido estudiado, quedan solamente 2 targets posibles: ENSG00000111726 y ENSG00000164414.

**En este caso, vamos a analizar el gen "ENSG00000164414"**

In [21]:
KMS20.genes.ENSG00000164414

Gene identifier,ENSG00000164414
Name,
Memory address,0x07fc5ffa712e8
Functional,True
In 2 reaction(s),"MAR07693, MAR04535"


In [28]:
KMS20.reactions.MAR07693

Reaction identifier,MAR07693
Name,
Memory address,0x07fc5fa04f2b0
Stoichiometry,MAM01590g[g] + MAM01592c[c] <=> MAM01590c[c] + MAM01592g[g] CMP [Golgi apparatus] + CMP-N-acetylneuraminate [Cytosol] <=> CMP [Cytosol] + CMP-N-acetylneuraminate [Golgi apparatus]
GPR,ENSG00000164414
Lower bound,-1000.0
Upper bound,1000.0


In [29]:
KMS20.reactions.MAR04535

Reaction identifier,MAR04535
Name,
Memory address,0x07fc5fa0e5b70
Stoichiometry,MAM01592c[c] <=> MAM01592n[n] CMP-N-acetylneuraminate [Cytosol] <=> CMP-N-acetylneuraminate [Nucleus]
GPR,ENSG00000164414
Lower bound,-1000.0
Upper bound,1000.0


Viendo las reacciones en las que participa, podemos intuir que se trata de un transportador de CMP, encargándose de transportar dicha molécula entre diferentes orgánulos tales como el Núcleo, el Aparato de Golgi o el Citosol.

Si comprobamos la eficacia de este gen como target en las 3 líneas de mieloma, obtenemos lo siguiente:

In [30]:
target = "ENSG00000164414"
with JJN3:
    JJN3.genes.get_by_id(target).knock_out()
    sol1=JJN3.slim_optimize()
with KMS20:
    KMS20.genes.get_by_id(target).knock_out()
    sol2=KMS20.slim_optimize()
with HUNS1:
    HUNS1.genes.get_by_id(target).knock_out()
    sol3=HUNS1.slim_optimize()
print(target,sol1,sol2,sol3)

ENSG00000164414 153.9552976499519 0.0 148.8926320607762


Vemos como la supresión de este gen solo tiene efecto en la línea "KMS20", como era de esperar. 